# **Automated Fuzzy Matching to Merge Similar Customer**

This code is processing a CSV file (POTENTIAL CUSTOMERS PDT DATA.csv) that contains customer data and aims to merge rows where customer names are similar, based on fuzzy matching.

Installing fuzzy wuzzy

In [ ]:
pip install fuzzywuzzy


Installing levenshtein

In [ ]:
pip install python-Levenshtein


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 20.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Objective:** To consolidate customer data by merging entries with similar customer names, effectively cleaning the dataset.

**Methodology:**

1. Uses fuzzy string matching to identify customer names that are similar
(accounting for typos or variations).

2. Merges the 'ASSESS VAL' by summing them up for similar customers.
3. Concatenates 'HSN DESC' descriptions for a comprehensive overview.
**Outcome:** A cleaned and consolidated dataset where each unique customer (after accounting for name variations) has a single entry, with their assessment values summed and descriptions combined.

**Note**: *the data was primarily cleaned and filtered using power query to set time frames, geographical locations.*

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Read the CSV file
filename = "/content/POTENTIAL CUSTOMERS PDT DATA.csv"
df = pd.read_csv(filename, encoding='ISO-8859-1')

# Handle duplicate columns with different names
df = df.loc[:, ~df.columns.duplicated()]

# Convert 'CUSTOMER' column to strings
df['CUSTOMER'] = df['CUSTOMER'].astype(str)

# Define a function to merge similar customer names
def merge_similar_names(df):
    # Dictionary to store indices of rows with similar names
    similar_names_dict = {}

    # Iterate over each row
    for i, row in df.iterrows():
        # Get the customer name from the current row
        name = row['CUSTOMER']

        # Find the most similar name in the dictionary
        similar_name_tuple = process.extractOne(name, similar_names_dict.keys(), scorer=fuzz.token_sort_ratio)

        # Check if a similar name was found
        if similar_name_tuple is not None:
            similar_name, score = similar_name_tuple

            # If the similarity score is above the threshold, merge the data
            if score > 80:
                # Add the current row index to the list of indices for the similar name
                similar_names_dict[similar_name].append(i)
            else:
                # Add a new entry in the dictionary for this name
                similar_names_dict[name] = [i]
        else:
            # Add a new entry in the dictionary for this name
            similar_names_dict[name] = [i]

    # Merge rows with similar names
    merged_rows = []
    for similar_name, indices in similar_names_dict.items():
        if len(indices) > 1:
            # Merge rows with similar names
            merged_row = df.iloc[indices[0]].copy()
            for index in indices[1:]:
                merged_row['ASSESS VAL'] += df.iloc[index]['ASSESS VAL']
                # Convert 'HSN DESC' values to string before concatenating
                merged_row['HSN DESC'] = str(merged_row['HSN DESC']) + ' ' + str(df.iloc[index]['HSN DESC'])
            merged_rows.append(merged_row)
        else:
            # If only one row, append as is
            merged_rows.append(df.iloc[indices[0]])

    # Create a new DataFrame with merged rows
    merged_df = pd.DataFrame(merged_rows)

    return merged_df

# Merge similar names
merged_df = merge_similar_names(df)

# Reset the index of the DataFrame
merged_df.reset_index(drop=True, inplace=True)

# Export the result in a new CSV file
merged_df.to_csv("result.csv", index=False)

print("Merging completed. Check 'result.csv' for the merged data.")


Merging completed. Check 'result.csv' for the merged data.
